In [ ]:
# 라이브러리

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

re : 정규표현 (Regular Expression)

In [ ]:
# data 파일 불러오기

train = pd.read_csv("train.csv")
test = pd.read_csv("test_x.csv")
sample_submission = pd.read_csv('sample_submission.csv')

**결측치 제거** (영어, 숫자만 남김)

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

**nltk 라이브러리** - tokenize & 불용어(stopword) 처리

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("stopwords")
nltk.download("punkt")
stopwords_list = stopwords.words('english')

In [ ]:
# train data

word_tokens = word_tokenize(train_clear_text) # 결측치 제거한 data
X_train = []

for word in word_tokens:
    if word not in stopwords_list:
        X_train.append(word)

In [ ]:
# test data

word_tokens = word_tokenize(test_clear_text) # 결측치 제거한 data
X_test = []

for word in word_tokens:
    if word not in stopwords_list:
        X_test.append(word)

**Tokenizer**

In [ ]:
# max_words = 224
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train_vec = tokenizer.texts_to_sequences(X_train)
X_test_vec = tokenizer.texts_to_sequences(X_test)

**author One-Hot Encoding** (keras 이용한 to_categorical() 활용 가능)

In [ ]:
# 토큰화한 문장 입력, 토큰 index 설정

word_to_index = {word : index for index, word in enumerate(tokens)}
print('단어 집합 :',word_to_index)

In [ ]:
# 토큰 입력하면 해당 토큰에 대한 원-핫 벡터 만드는 함수
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

**to_categorical()** 예시

In [ ]:
from tensorflow.keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print('단어 집합 :',tokenizer.word_index)

In [ ]:
encoded = tokenizer.texts_to_sequences(X_train)[0]
print(encoded)

In [ ]:
one_hot = to_categorical(encoded)
print(one_hot)

**패딩(Padding)** pad_sequences

In [ ]:
#max_len = 21

X_train_vec = pad_sequences(X_train_vec, maxlen=max_len)
X_test_vec = pad_sequences(X_test_vec, maxlen=max_len)

**모델 학습**

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_vec, y_train, epochs=10, batch_size=32, validation_split=0.1)